# Mar ??, 2022 ()

**Motivation**:  <br>

In [1]:
# HIDE CODE


import os
import sys
import nibabel as nib
import networkx as nx
from time import time
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from numpy.ma import masked_where as mwh
from scipy.ndimage import gaussian_filter
from IPython.display import display, IFrame, HTML
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.pyplot as plt
import seaborn as sns

# set style & no interpolalation
import matplotlib
matplotlib.rcParams['image.interpolation'] = 'none'
sns.set_style('whitegrid')
%matplotlib inline

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')
lfr_dir = pjoin(os.environ['HOME'], 'Documents/workspaces/lfr/binary_overlapping')


# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from register.atlas import make_tree_graph
from register.parcellation import Parcellation
from analysis.hierarchical import Hierarchical
from analysis.fourier import *
from analysis.bootstrap import *
from analysis.svinet import *
from analysis.group import *
from analysis.lfr import *
from utils.render import *
from utils.plotting import *
from model.mouse import Mice
from model.configuration import Config

# warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Fig dir

In [2]:
fig_dir = pjoin(tmp_dir, 'fig_paper')
os.makedirs(fig_dir, exist_ok=True)

## Prepare

In [3]:
mice = Mice(128)
mice.setup_func_data('rest')

mako = sns.color_palette('mako', as_cmap=True)
rocket = sns.color_palette('rocket', as_cmap=True)
pal = {'ca2': mako(0.6), 'bold-lite': rocket(0.6)}

num_k, perc = 5, 10
props_base = {
    'perc': f'p{perc}-sample',
    'num_k': num_k,
    'mice': mice,
    'task': 'rest',
    'metric': 'pearson',
    'match_using': 'gam',
    'match_metric': 'euclidean',
    'graph_type': 'real',
    'verbose': False,
}
props_ca = {'mode': 'ca2', **props_base}
props_bold = {'mode': 'bold-lite', **props_base}

In [7]:
gr = Group(**props_bold).load_group()

In [9]:
x = np.load(pjoin(gr.path, 'run_gam.npy'), mmap_mode='r')
pi = pi_from_gam(x)
pi = bn.nanmean(bn.nanmean(pi, 2), 1)
x.shape, pi.shape

((10, 3, 4, 500, 5, 154), (10, 5, 154))

In [10]:
%%time

i, j, k = x.shape[:3]
dims = {'ses': j, 'run': k}
# dims = {'sub': i, 'ses': j, 'run': k}

n_resamples = 300

b_anim = bootstrap(
    x=x[6],
    n_resamples=n_resamples,
    batch=5,
    # skip=['sub'],
    dims=dims,
    est=bn.nanmean,
)
b_anim.shape

CPU times: user 6.13 s, sys: 3.04 s, total: 9.18 s
Wall time: 9.18 s


(300, 3, 4, 5, 154)

In [11]:
b_anim = pi_from_gam(b_anim)
b_anim = bn.nanmean(bn.nanmean(b_anim, 2), 1)
se_anim = np.nanstd(b_anim, axis=0, ddof=1)

In [12]:
sns.histplot(b_anim[:, 2, 17])
plt.xlim(0, 1);

In [13]:
ci = 0.99
whis = (1 + ci) / 2
t = sp_stats.t.ppf(whis, 11)
t

3.105806513221101

In [14]:
for i in range(5):
    lo = pi[6, i, 17] - se_anim[i, 17] * t
    print(i, lo, lo < 0.01)

0 0.03544937061737001 False

1 -0.020040802915328497 True

2 0.4210505736073707 False

3 -0.08937976267994166 True

4 -0.010637726701054041 True

In [16]:
pi[6, :, 17]

array([0.15788363, 0.0211015 , 0.69140685, 0.08041056, 0.04919745])